<a href="https://colab.research.google.com/github/cincinnatilibrary/collection-analysis/blob/master/reports/count_item_types_by_branch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **CHPL - Collection Analysis**

### Count Item Types By Branch (status "Available", not checkted out)

<img src="https://ilsweb.cincinnatilibrary.org/img/CHPL_Brandmark_Primary.png" alt="CHPL" title="CHPL" width="300"/>

Data sources

* https://collection-analysis.cincy.pl/

In [2]:
#@title
import pandas as pd
import requests
import altair as alt
import re

base_url = 'https://collection-analysis.cincy.pl'

# find the name of the target current_collection db--since it's named with its 
# hashed value
# https://datasette.io/plugins/datasette-hashed-urls
db_available = requests.get(
      url=base_url + '/-/databases' + '.json',
).json()

db_name = ''

for db in db_available:
  if re.search(r"^current_collection.*$", db['name']):
    db_name = db['name']

print(db_name)

sql = """\
select
  branch_name,
  item_format,
  count(item_record_num) as count_item_record_num
from
  item_view
where
  "due_date" is null
  and branch_name is not null
  and "item_status_code" = :p0
group by
  1,
  2
"""

df = pd.read_json(
    requests.get(
      url=base_url + '/' + db_name + '.json',
      params={
        'sql': sql,
        'p0': '-',
        '_shape': 'array'
      }
    ).content
)



current_collection


In [3]:
#@title
# generate the chart using Altair
# https://altair-viz.github.io/user_guide/encoding.html
chart = alt.Chart(df).mark_bar(color='#34B78F').encode(
    y = alt.Y(
        'branch_name',
        # axis=alt.Axis(labelAngle=-70)
    ),
    x = alt.X(
        'count_item_record_num:Q',
    ),
    color='item_format',
    tooltip=['branch_name', 'item_format', 'count_item_record_num']
).properties(
    title="Count CHPL Titles by Decade of Publication",
    width=900
)
chart.display()

alt.Chart(...)

#### Excluding Main, Outreach Services ...



In [4]:
#@title
# generate the chart using Altair
# https://altair-viz.github.io/user_guide/encoding.html
chart = alt.Chart(df[~df['branch_name'].isin(['Main Library', 'Outreach Services' ])]).mark_bar(color='#34B78F').encode(
    y = alt.Y(
        'branch_name',
        # axis=alt.Axis(labelAngle=-70)
    ),
    x = alt.X(
        'count_item_record_num:Q',
    ),
    color='item_format',
    tooltip=['branch_name', 'item_format', 'count_item_record_num']
).properties(
    title="Count CHPL Titles by Decade of Publication",
    width=900
)
chart.display()

alt.Chart(...)

#### Heatmap of Count Items by "item_type" and "branch_name" Excluding Main, Outreach Services ...

In [11]:
#@title
alt.Chart(df[~df['branch_name'].isin(['Main Library', 'Outreach Services' ])]).mark_rect().encode(
    # 
    x=alt.X(
        'branch_name:O',
        axis=alt.Axis(labelAngle=-70)
    ),
    y='item_format:O',
    color=alt.Color(
        'count_item_record_num:Q', 
        scale=alt.Scale(scheme='greens')
    ),
    tooltip=['branch_name', 'item_format', 'count_item_record_num']
).properties(
    title="Heatmap of Count Items by \"item_type\" and \"branch_name\" Excluding Main, Outreach Services",
    width=900,
    height=1000
)

alt.Chart(...)